# Assignment 5

## 1.复习课上内容， 阅读相应论文。

## 2. 回答以下理论题目

### 2. 1.  What is autoencoder?

- 把输入通过一个网络压缩到隐空间，再由另一个网络还原。然后将输出与原始输入做比较，求他们的距离，优化这个距离，达到两者相似甚至一致的效果。
   autoencoder是一种数据的压缩算法，其中数据的压缩和解压缩函数是数据相关的、有损的、从样本中自动学习的。
   在大部分提到自动编码器的场合，压缩和解压缩的函数是通过神经网络实现的。

### 2. 2. What are the differences between greedy search and beam search?

- greedy search每个节点都寻找最优的词，beam search会考虑多个选择从而综合考虑多个词的效果整体。


### 2. 3. What is the intuition of attention mechanism?

- 进行局部聚焦，长程联系。attention mechanism是一种用于提升基于RNN（LSTM或GRU）的Encoder + Decoder模型的效果的的机制。
   Attention Mechanism可以帮助模型对输入的X每个部分赋予不同的权重，抽取出更加关键及重要的信息，使模型做出更加准确的判断，
   同时不会对模型的计算和存储带来更大的开销。

### 2. 4. What is the disadvantage of word embeding introduced in previous lectures ?

- 不能解决一词多义问题。

### 2. 5. Briefly describe what is self-attention and what is multi-head attention?

- 当模型在处理每个单词时，self-attention 可以帮助模型查看 input 序列中的其他位置，寻找相关的线索，来达到更好的编码效果。
   它的作用就是将对其他相关单词的“understanding”融入我们当前正在处理的单词中。
   self-attention 具体原理：
   第一步，为编码器的每个输入单词创建三个向量，
   第二步，是计算一个得分，
   第三步和第四步，是将得分加以处理再传递给 softmax
   第五步，用这个得分乘以每个 value 向量
   第六步，加权求和这些 value 向量

## 3. 中英文自动翻译模型的构建（使用encoder-decoder模型）

![](https://media.geeksforgeeks.org/wp-content/uploads/seq2seq.png)

### 3.1 [中英文翻译数据集下载](http://www.manythings.org/anki/)
找到Chinese (Mandarin) - English cmn-eng.zip (22075条中英文翻译)

### 3.2  数据处理：encoder的输入，decoder的输入与输出
1，句子转换为one-hot编码     
2，LSTM需要的三维输入[n_samples, timestamp, one-hot feature]

In [3]:
from keras.layers import Input, LSTM, Dense
from keras.models import Model, load_model
from keras.utils import plot_model
import pandas as pd
import numpy as np

N_UNITS = 256
BATCH_SIZE = 64
EPOCH = 50
NUM_SAMPLES = 10000


data_path = 'cmn.txt'
df = pd.read_table(data_path,header=None).iloc[:NUM_SAMPLES,:,]
df.columns=['inputs', 'targets', 'others']

df['targets'] = df['targets'].apply(lambda x: '\t'+x+'\n')

input_texts = df.inputs.values.tolist()
target_texts = df.targets.values.tolist()

input_characters = sorted(list(set(df.inputs.unique().sum())))
target_characters = sorted(list(set(df.targets.unique().sum())))

INUPT_LENGTH = max([len(i) for i in input_texts])
OUTPUT_LENGTH = max([len(i) for i in target_texts])
INPUT_FEATURE_LENGTH = len(input_characters)
OUTPUT_FEATURE_LENGTH = len(target_characters)

encoder_input = np.zeros((NUM_SAMPLES, INUPT_LENGTH, INPUT_FEATURE_LENGTH))
decoder_input = np.zeros((NUM_SAMPLES, OUTPUT_LENGTH, OUTPUT_FEATURE_LENGTH))
decoder_output = np.zeros((NUM_SAMPLES, OUTPUT_LENGTH, OUTPUT_FEATURE_LENGTH))

input_dict = {char:index for index,char in enumerate(input_characters)}
input_dict_reverse = {index:char for index,char in enumerate(input_characters)}
target_dict = {char:index for index,char in enumerate(target_characters)}
target_dict_reverse = {index:char for index,char in enumerate(target_characters)}

for seq_index,seq in enumerate(input_texts):
    for char_index, char in enumerate(seq):
        encoder_input[seq_index, char_index, input_dict[char]] = 1

for seq_index,seq in enumerate(target_texts):
    for char_index,char in enumerate(seq):
        decoder_input[seq_index,char_index, target_dict[char]] = 1.0
        if char_index > 0:
            decoder_output[seq_index,char_index-1, target_dict[char]] = 1.0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [5]:
print(encoder_input[0][1])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### 3.3 encoder-decoder模型的搭建
1，模型训练    
2，模型推理     
3，模型预测，展示结果   

In [6]:
def create_model(n_input, n_output, n_units):
    # encoder
    encoder_input = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    _,encoder_h, encoder_c = encoder(encoder_input)
    encoder_state = [encoder_h, encoder_c]
    
    
    decoder_input = Input(shape=(None, n_output))
    decoder = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_output, _, _ = decoder(decoder_input,
                                   initial_state=encoder_state)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_output = decoder_dense(decoder_output)
    
    model = Model([encoder_input, decoder_input], decoder_output)
    
    encoder_infer = Model(encoder_input, encoder_state)
    
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))    
    decoder_state_input = [decoder_state_input_h, decoder_state_input_c] 
    
    decoder_infer_output, decoder_infer_state_h, decoder_infer_state_c = decoder(decoder_input,
                                                                                 initial_state=decoder_state_input)
    decoder_infer_state = [decoder_infer_state_h, decoder_infer_state_c]
    decoder_infer_output = decoder_dense(decoder_infer_output)
    decoder_infer = Model([decoder_input] + decoder_state_input,
                          [decoder_infer_output] + decoder_infer_state)
    
    return model, encoder_infer, decoder_infer


def predict_chinese(source,encoder_inference, decoder_inference, n_steps, features):
    state = encoder_inference.predict(source)
    predict_seq = np.zeros((1,1,features))
    predict_seq[0,0,target_dict['\t']] = 1
    output = ''

    for i in range(n_steps):
        yhat,h,c = decoder_inference.predict([predict_seq]+state)
        char_index = np.argmax(yhat[0,-1,:])
        char = target_dict_reverse[char_index]
        output += char
        state = [h,c]
        predict_seq = np.zeros((1,1,features))
        predict_seq[0,0,char_index] = 1
        if char == '\n':
            break
    return output

In [8]:
model_train, encoder_infer, decoder_infer = create_model(
    INPUT_FEATURE_LENGTH,
    OUTPUT_FEATURE_LENGTH,
    N_UNITS)

model_train.compile(optimizer='rmsprop', loss='categorical_crossentropy')

validation_split = 0.2
model_train.fit([encoder_input,decoder_input],
    decoder_output,
    batch_size=BATCH_SIZE,
    epochs=EPOCH,
    validation_split=validation_split)

Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 [==============================] - 111s 14ms/step - loss: 1.9406 - val_loss: 2.4999
Epoch 2/50
8000/8000 [==============================] - 110s 14ms/step - loss: 1.8125 - val_loss: 2.3824
Epoch 3/50
8000/8000 [==============================] - 103s 13ms/step - loss: 1.7147 - val_loss: 2.2488
Epoch 4/50
8000/8000 [==============================] - 99s 12ms/step - loss: 1.6255 - val_loss: 2.1783
Epoch 5/50
8000/8000 [==============================] - 98s 12ms/step - loss: 1.5354 - val_loss: 2.1185
Epoch 6/50
8000/8000 [==============================] - 99s 12ms/step - loss: 1.4665 - val_loss: 2.0797
Epoch 7/50
8000/8000 [==============================] - 97s 12ms/step - loss: 1.4005 - val_loss: 1.9985
Epoch 8/50
8000/8000 [==============================] - 102s 13ms/step - loss: 1.3460 - val_loss: 1.9487
Epoch 9/50
8000/8000 [==============================] - 98s 12ms/step - loss: 1.2944 - val_loss: 1.9112
Epoch 10/50


In [10]:
for i in range(1000,1100):
    test = encoder_input[i:i+1,:,:] 
    out = predict_chinese(test,encoder_infer,decoder_infer,OUTPUT_LENGTH,OUTPUT_FEATURE_LENGTH)
    print(input_texts[i])
    print(out)
    

Stop grumbling.
别再发生了。

Stop resisting!
别再发生了。

Summer is over.
有人在這裡。

Take your time.
你可以慢慢来。

Take your time.
你可以慢慢来。

That was wrong.
那真好嗎？

That's a shame.
那是我的。

That's logical.
那是我的。

That's my coat.
那是我的。

That's perfect.
那是我的。

That's too bad.
那是我的。

That's too bad.
那是我的。

That's too bad.
那是我的。

The birds sang.
天空很晴朗。

The flag is up.
這隻狗很大。

The phone rang.
這裡的工作很棒。

Their eyes met.
這裡的書一本書。

These are pens.
這隻狗很大。

They hated Tom.
他們有工作。

They have jobs.
他们是我的。

They let me go.
他们是我的。

They love that.
他们是孩子。

They trust Tom.
他们是我的。

They want more.
他們是我們的兄弟。

They want this.
他們是我們的蘋果。

They were good.
他们是我的。

This is a book.
這是我的書。

This is my bag.
這是我的書。

Tom can change.
汤姆不会游泳。

Tom can't swim.
湯姆不會停。

Tom has a plan.
湯姆是個好工人。

Tom is a rabbi.
湯姆是個好工人。

Tom is no fool.
湯姆不是個好。

Tom isn't dumb.
湯姆不是個好。

Tom looks pale.
湯姆喜歡做。

Tom loves dogs.
湯姆喜歡看。

Tom turned red.
汤姆睡着了。

Tom walked out.
湯姆會等。

Tom was crying.
湯姆不會開車。

Tom won't stop.
湯姆不會停。

Tom's fearless.
汤姆在笑。

Tom's 

![](https://stickershop.line-scdn.net/stickershop/v1/product/3624648/LINEStorePC/main.png;compress=true)